In [8]:
import sys

def kw_defaults(kw):
    def helper(key, val):
        return val if key not in kw.keys() else kw[key]
    return helper

def get_arg(key, val, f=str):
    s = ' '.join(sys.argv)
    t = '%s='%key
    if( t in s ):
        return f(s.split(t)[-1].split(' ')[0])
    else:
        return val


In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import minimize
from typing import Callable

import numpy as np

def gd(**kw):
    kwd = kw_defaults(kw)

    #REQUIRED FIELDS
    f = kw['f']
    g = kw['grad']
    x0 = kw['x0']

    #OPTIONAL FIELDS
    tau = kwd('tau', 1e-15)
    c_armijo = kwd('c_armijo', 1e-4)
    c_curvature = kwd('c_curvature', 1e-2)
    alpha = kwd('alpha', 1.0)
    mode = kwd('mode', 'backtracking')
    max_iter = kwd('max_iter', 100)
    verbose = kwd('verbose', True)
    stochastic = kwd('stochastic', lambda x,it: np.zeros(x0.shape))

    #Setup iteration variables
    d = len(x0)
    x = x0
    beta = alpha

    #Setup return values
    x_history = [x]
    f_history = [f(x)]
    g_history = [g(x)]
    g_norm_history = [np.linalg.norm(g_history[-1])]
    r_history = []

    sep = '     '
    print('it: 0%sx: [%s]%sf: %.5e%sgrad_norm: %.5e%sgrad: [%s]%sstep_length: %.5e'%(
        sep,
        ','.join(['%.5e'%e for e in x]), sep,
        f_history[-1], sep,
        g_norm_history[-1], sep,
        ','.join(['%.5e'%e for e in g_history[-1]]), sep,
        beta))

    it = 0
    while(it < max_iter):
        phi = lambda gamma : f(x - gamma * g(x))
        # Derivative of phi with respect to gamma
        phi_prime = lambda gamma : -np.dot(g(x - gamma * g(x)), g(x))

        # Armijo condition: f(x_new) ≤ f(x) - c₁ α ∇f(x)ᵀp
        armijo = lambda gamma : phi(gamma) <= phi(0) - c_armijo * gamma * np.dot(g(x), g(x))

        # Curvature condition: -∇f(x_new)ᵀp ≥ c₂ ∇f(x)ᵀp
        curv = lambda gamma : -phi_prime(gamma) <= -c_curvature * phi_prime(0)

        beta = alpha

        if(mode == 'backtracking'):
            # Check if either Wolfe condition is violated
            while(not armijo(beta) or not curv(beta)):
                # Backtracking step: reduce step size
                beta *= 0.5
                if beta < 1e-10:  # Prevent step size from becoming too small
                    print("Warning: Step size became too small")
                    break

        elif(mode == 'exact'):
            #Exact line search done...note that we have
            #    call an external package to solve an auxiliary opt. problem.
            tmp = minimize(phi, 0.0)
            beta = tmp['x'][0]

        # Update step with stochastic term
        # x_new = x_current - step_size * gradient + stochastic_term
        x = x - beta * g(x) + stochastic(x, it)

        #Update loop variables
        x_history.append(x)
        f_history.append(f(x))
        g_history.append(g(x))
        g_norm_history.append(np.linalg.norm(g_history[-1]))
        r_history.append(abs(f_history[-2] - f_history[-1]))

        it += 1

        #Print debug info
        if(verbose):
            print('it: %d%sx: [%s]%sf: %.5e%sgrad_norm: %.5e%sgrad: [%s]%sstep_length: %.5e%sr: %.5e'%(
                it, sep,
                ','.join(['%.5e'%e for e in x]), sep,
                f_history[-1], sep,
                g_norm_history[-1], sep,
                ','.join(['%.5e'%e for e in g_history[-1]]), sep,
                beta,sep,
                r_history[-1]))

        if(g_norm_history[-1] < tau):
            print('Gradient norm = %.5e < %.5e...breaking'%(np.linalg.norm(g(x)), tau))
            break

    #return values
    return {'x': x_history[-1],
            'val': f_history[-1],
            'x_history': x_history,
            'f_history': f_history,
            'grad_norm_history': g_norm_history,
            'grad_history': g_history,
            'residual': r_history,
            'iterations': it}

#Make plots -- no modifications needed
"""
def plot_progress(f, x_history, output_name, x_box, y_box, num_levels):
    if( len(x_history) == 0 ):
        raise(Exception('Empty history list...cannot plot'))

    if( len(x_history[0]) > 2 ):
        raise(Exception('Domain of dimension %d > 2...cannot plot'%(len(x_history[0]))))

    if( len(x_history[0]) == 1 ):
        x_history = np.ndarray.flatten(np.array(x_history))
        x_min = min(x_history)
        x_max = max(x_history)
        x = np.array([np.array([e]) for e in np.linspace(x_min, x_max, N)])
        colors = cm.rainbow(np.linspace(0,1,len(x_history)))
        for xx, c in zip(x_history, colors):
            plt.scatter( xx, f(np.array([xx])), color=c )
        plt.plot(np.ndarray.flatten(x),np.array(list(map(f,x))))
        plt.xlabel('X')
        plt.ylabel('f')
        plt.savefig('%s.pdf'%output_name.replace('.pdf',''))
    else:
        x_coords = [e[0] for e in x_history]
        y_coords = [e[1] for e in x_history]

        X,Y = np.meshgrid(x_box,y_box)

        Z = np.zeros(X.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = f(np.array([X[i][j], Y[i][j]]))

        bar = plt.contourf(X,Y,Z, num_levels)
        plt.colorbar(bar)

        colors = cm.rainbow(np.linspace(0,1,len(x_history)))
        for xx, c in zip(x_history, colors):
            plt.scatter( xx[0], xx[1], color=c )
        plt.title(output_name.replace('.pdf',''))
        plt.savefig('%s.pdf'%(output_name.replace('.pdf','')))

"""
def plot_progress(f, x_history, output_name, x_box, y_box, num_levels):
    if(len(x_history) == 0):
        raise(Exception('Empty history list...cannot plot'))

    if(len(x_history[0]) > 2):
        raise(Exception('Domain of dimension %d > 2...cannot plot'%(len(x_history[0]))))

    if(len(x_history[0]) == 1):
        x_history = np.ndarray.flatten(np.array(x_history))
        x_min = min(x_history)
        x_max = max(x_history)
        x = np.array([np.array([e]) for e in np.linspace(x_min, x_max, 1000)])
        colors = cm.rainbow(np.linspace(0, 1, len(x_history)))
        for xx, c in zip(x_history, colors):
            plt.scatter(xx, f(np.array([xx])), color=c)
        plt.plot(np.ndarray.flatten(x), np.array(list(map(f,x))))
        plt.xlabel('X')
        plt.ylabel('f')
    else:
        # Create the contour plot
        X, Y = np.meshgrid(x_box, y_box)
        Z = np.zeros(X.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = f(np.array([X[i][j], Y[i][j]]))

        plt.figure(figsize=(12, 8))
        # Plot contour
        bar = plt.contourf(X, Y, Z, num_levels, cmap='viridis')
        plt.colorbar(bar)

        # Plot optimization path with rainbow colors to show progression
        colors = cm.rainbow(np.linspace(0, 1, len(x_history)))
        for i, (xx, c) in enumerate(zip(x_history, colors)):
            plt.scatter(xx[0], xx[1], color=c, s=50)
            # Add arrows between consecutive points
            if i < len(x_history) - 1:
                dx = x_history[i+1][0] - xx[0]
                dy = x_history[i+1][1] - xx[1]
                plt.arrow(xx[0], xx[1], dx, dy,
                         head_width=0.2, head_length=0.3,
                         fc=c, ec=c, alpha=0.5)

        # Highlight start and end points
        plt.scatter(x_history[0][0], x_history[0][1], color='purple',
                   s=100, label='Start', marker='*')
        plt.scatter(x_history[-1][0], x_history[-1][1], color='red',
                   s=100, label='End', marker='*')

        plt.legend()
        plt.title(f'Optimization Path ({len(x_history)} steps)')
        plt.xlabel('x₁')
        plt.ylabel('x₂')

    plt.savefig(f'{output_name}.pdf')
    plt.close()
#"""




In [16]:
import numpy as np

def rastrigin(x):
    """
    Implements the n-dimensional Rastrigin function:
    f(x) = An + sum(x_i^2 - A*cos(2π*x_i)) where A = 10

    Parameters:
    x (np.ndarray): Input vector of length n

    Returns:
    float: Function value at x
    """
    A = 10.0
    n = len(x)
    return A*n + np.sum(x**2 - A*np.cos(2*np.pi*x))

def rastrigin_gradient(x):
    """
    Implements the gradient of the n-dimensional Rastrigin function:
    ∇f(x)_i = 2x_i + 2πA*sin(2π*x_i) where A = 10

    Parameters:
    x (np.ndarray): Input vector of length n

    Returns:
    np.ndarray: Gradient vector at x
    """
    A = 10.0
    return 2*x + 2*np.pi*A*np.sin(2*np.pi*x)

# Update your script with:
f = rastrigin
g = rastrigin_gradient

In [17]:
A = 10.0
n = 2

#Setup initial condition
x0 = get_arg('x_init', np.array([25.0, 60.0]), lambda x : np.array(eval(x)))
d = len(x0)

#TODO: Tune parameters for better convergence! Try higher dimensions!
#tolerance level
tau = get_arg('tau', 1e-7, float)

#armijo constant
c_armijo = get_arg('c_armijo', 1e-4, float)

#curvature constant -- note that it must be greater than armijo constant!
c_curvature = get_arg('c_curvature', 1e-2, float)

#max iterations
max_iter = get_arg('max_iter', 1000, int)

#mode -- either "exact" or "backtracking"
mode = get_arg('mode', 'exact', str)

#stochastic terms
sig_max = get_arg('sig_max', 0.1, float)
sig_min = get_arg('sig_min', 0.01, float)
output_file = get_arg('output_file', 'output', str)

#TODO: modify rate of decay of stochastic term with iteration number
stochastic = lambda x,it : np.random.randn(d) * ( (max_iter-it) * sig_max + it * sig_min )

#run program and get output
d = gd(f=f, grad=g, x0=x0,
    tau=tau,
    c_armijo=c_armijo,
    c_curvature=c_curvature,
    max_iter=max_iter,
    mode=mode,
    stochastic=stochastic)

#print final results
print('Approx min of %f at %s after %d iterations'%(d['val'], d['x'], d['iterations']))

#define plotting window
x_box = np.linspace(-10,10,1000)
y_box = np.linspace(-10,10,1000)
num_levels = 50

#plot, if possible
try:
    plot_progress(f, d['x_history'], output_file, x_box, y_box, num_levels)
except Exception as e:
    print(e)

it: 0     x: [2.50000e+01,6.00000e+01]     f: 4.22500e+03     grad_norm: 1.30000e+02     grad: [5.00000e+01,1.20000e+02]     step_length: 1.00000e+00
it: 1     x: [1.03515e+02,-6.13809e+01]     f: 1.45203e+04     grad_norm: 2.60404e+02     grad: [2.01024e+02,-1.65528e+02]     step_length: 5.17015e-01     r: 1.02953e+04
it: 2     x: [2.54975e+01,2.04906e+02]     f: 4.26583e+04     grad_norm: 3.78416e+02     grad: [5.19751e+01,3.74829e+02]     step_length: 4.65826e-01     r: 2.81380e+04
it: 3     x: [-6.03348e+01,-1.43891e+02]     f: 2.43623e+04     grad_norm: 3.03518e+02     grad: [-1.74793e+02,-2.48134e+02]     step_length: 4.85749e-01     r: 1.82960e+04
it: 4     x: [1.05066e+02,1.42823e+02]     f: 3.14437e+04     grad_norm: 3.28676e+02     grad: [2.35420e+02,2.29359e+02]     step_length: 4.94901e-01     r: 7.08142e+03
it: 5     x: [1.67333e+02,-3.97853e+01]     f: 2.96061e+04     grad_norm: 3.89518e+02     grad: [3.89089e+02,-1.82791e+01]     step_length: 5.44044e-01     r: 1.83764e+